Kevin Lin

Data 602, Homework 2


# Import required modules

In [ ]:
import zipfile
import os
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from pathlib import Path
from PIL import Image

# Preprocess Data
First import the images and turn them into 1 dimensional vectors and create a list of their corresponding labels from the file names.

In [ ]:
# Unzip the data set
file_path = '/content/Face Data for Homework.zip'
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Remove the text file contained in the folder
if os.path.exists('/content/ATT/README'):
    os.remove('/content/ATT/README')

# Read each image row by row and turn into a 1D list and then append to a 2D list of all the images for a subject
# Also get the Label for the image from the filename (ie. '10_2.png' is label_count.png) and use the label as the index in the list of lists
vector_images = []
vector_labels = []
for image_file in Path('/content/ATT').glob('*.png'):
    vector_labels.append(int(image_file.stem.split('_')[0]))
    with Image.open(image_file) as image:
      vector_images.append(np.array(image).flatten())

print("Images:", len(vector_images), "\nImage Vector Length:", len(vector_images[0]), "\nLabels:", len(vector_labels))
print("Label:", vector_labels[0], "| Image:", vector_images[0])

Images: 400 
Image Vector Length: 10304 
Labels: 400
Label: 33 | Image: [ 93  92  98 ... 118 119 130]


# Implement 5-Fold Cross-Validation (Stratified)
We need to shuffle the images within their corresponding labels so there the same amount of training and testing images for all 40 subjects. This will give 8 photos for training and 2 testing photos per person for a total of 320 training photos and 80 testing photos per fold.

---

Then for each fold implement the Eigenfaces (PCA) Method to prepare the dataset to be used for kNN where k = 1.

---

Test and display results for each fold. Then aggregate and average the results at the end (we are asked for average prediction accuracy which is TP/(TP+FP)).

In [ ]:
# Make our own kNN classifier
def ONEnn_predict(X_train, y_train, X_test):
    # Distance is Euclidean norm between test point and all of training set samples
    distances = np.linalg.norm(X_train - X_test, axis=1)

    # Sort the distances to find the lowest value/closest point
    sorted_indices = np.argsort(distances)

    # Get and return the label of the closest point
    return y_train[sorted_indices[0]]

In [ ]:
# Use the builtin StratifiedKFold to divide the folds
skf = StratifiedKFold(n_splits=5, shuffle=True)

# Convert vector_images and vector_labels into numpy arrays
vector_images = np.array(vector_images)
vector_labels = np.array(vector_labels)

# For each fold implement the Eigenfaces (PCA) Method, and then test on the testing dataset
results = []
for fold, (train_index, test_index) in enumerate(skf.split(vector_images, vector_labels), 1):
    # Split the data into training and testing sets
    X_train, X_test = vector_images[train_index], vector_images[test_index]
    y_train, y_test = vector_labels[train_index], vector_labels[test_index]

    # Perform PCA on the training data
    # X_train is 320x10304 2D list, I will end up with 320 possible principal components
    # I have to choose from within the range [1, 320]
    pca = PCA(n_components=40)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Loop through each transformed test point and their corresponding test label, tally the correct predictions
    correct_predictions = 0
    for test_point, test_label in zip(X_test_pca, y_test):
        # Predict the label for the test point
        y_pred = ONEnn_predict(X_train_pca, y_train, test_point)
        # Check if the prediction is correct
        if y_pred == test_label:
            correct_predictions += 1

    # Calculate the accuracy for this fold (TP + FP I realize is just the length of 1 fold)
    accuracy = correct_predictions / len(y_test)
    results.append(accuracy)

    # Display the results for each of the folds
    print(f"Fold {fold}: Accuracy = {accuracy}")


# Aggregate the results at the end
average_accuracy = np.mean(results)
print(f"Average Accuracy: {average_accuracy}")

Fold 1: Accuracy = 1.0
Fold 2: Accuracy = 0.975
Fold 3: Accuracy = 0.9625
Fold 4: Accuracy = 0.9625
Fold 5: Accuracy = 0.9875
Average Accuracy: 0.9775
